In [140]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage, trim_messages
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from typing import Sequence
from typing_extensions import Annotated, TypedDict

In [141]:
# # Load env variables
# load_dotenv()
# openai_key = os.getenv('OPENAI_API_KEY')

In [142]:
llm = ChatOpenAI(model='gpt-3.5-turbo')

In [143]:
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

In [144]:
trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

In [145]:
prompt = ChatPromptTemplate.from_messages([
    (
        'system',
        "you are a private tutor personalized for every student who is studying robotics, answer in a very intelligent way,  reply in {language}"
    ),
    MessagesPlaceholder(variable_name='messages'),
])

In [146]:
# Define the function that calls the model
def call_model(state: State):
    chain = prompt | llm
    trimmed_message = trimmer.invoke(state["messages"])
    response = chain.invoke({
        "messages" : trimmed_message,
        "language": state["language"]
    })
    return {'messages':[response]}

In [147]:
# define a new graph
workflow = StateGraph(state_schema = State)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

In [148]:
# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [149]:
config = {"configurable": {"thread_id": "abc123"}}

In [160]:
# config = {"configurable": {"thread_id": "abc123"}}
input_messages = [HumanMessage("whats my name")]
language = "English"
output = app.invoke(
    {"messages" : input_messages, "language" : language}, 
    config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm sorry, but as a private tutor, I don't have access to personal information about my students, including your name. How can I assist you today with your robotics studies?


In [162]:
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

|Good| day|,| dear| student|.| How| may| I| assist| you| in| your| studies| of| robotics| today|?||